<a href="https://colab.research.google.com/github/kashyap333/Playground_Series_Season_3-_Episode_3/blob/main/Playground_Series_Season_3%2C_Episode_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!kaggle competitions download -c playground-series-s3e3


  0% 0.00/95.2k [00:00<?, ?B/s]
100% 95.2k/95.2k [00:00<00:00, 56.4MB/s]


In [3]:
!unzip playground-series-s3e3

Archive:  playground-series-s3e3.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [28]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [29]:
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,...,80,1,10,2,3,10,0,7,8,0
1,1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,...,80,1,4,3,3,4,2,0,3,0
2,2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,...,80,2,4,3,3,3,2,1,2,0
3,3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,...,80,0,15,1,1,6,0,0,2,0
4,4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,...,80,0,31,0,3,31,14,4,10,1


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677 entries, 0 to 1676
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        1677 non-null   int64 
 1   Age                       1677 non-null   int64 
 2   BusinessTravel            1677 non-null   object
 3   DailyRate                 1677 non-null   int64 
 4   Department                1677 non-null   object
 5   DistanceFromHome          1677 non-null   int64 
 6   Education                 1677 non-null   int64 
 7   EducationField            1677 non-null   object
 8   EmployeeCount             1677 non-null   int64 
 9   EnvironmentSatisfaction   1677 non-null   int64 
 10  Gender                    1677 non-null   object
 11  HourlyRate                1677 non-null   int64 
 12  JobInvolvement            1677 non-null   int64 
 13  JobLevel                  1677 non-null   int64 
 14  JobRole                 

In [8]:
train.select_dtypes(include='object').columns

Index(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole',
       'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')

In [9]:
train.select_dtypes(include='int').columns

Index(['id', 'Age', 'DailyRate', 'DistanceFromHome', 'Education',
       'EmployeeCount', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome',
       'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Attrition'],
      dtype='object')

In [30]:
train = train.drop('id', axis=1)

In [12]:
train.Department.unique()

array(['Research & Development', 'Sales', 'Human Resources'], dtype=object)

###Encoding

In [31]:
enc = Pipeline(steps=[
    ('encoder', OrdinalEncoder())
])

In [32]:
train[train.select_dtypes(include='object').columns] = enc.fit_transform(train.select_dtypes(include='object'))

In [33]:
train.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,36,1.0,599,1.0,24,3,3.0,1,4,1.0,...,80,1,10,2,3,10,0,7,8,0
1,35,2.0,921,2.0,8,3,4.0,1,1,1.0,...,80,1,4,3,3,4,2,0,3,0
2,32,2.0,718,2.0,26,3,2.0,1,3,1.0,...,80,2,4,3,3,3,2,1,2,0
3,38,2.0,1488,1.0,2,3,3.0,1,3,0.0,...,80,0,15,1,1,6,0,0,2,0
4,50,2.0,1017,1.0,5,4,3.0,1,2,0.0,...,80,0,31,0,3,31,14,4,10,1


In [36]:
train_target = train['Attrition']
train_data = train.drop('Attrition', axis=1)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size = 0.1, random_state=42)

In [47]:
clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.001)

In [48]:
model = clf.fit(X_train, y_train)

In [49]:
y_pred = model.predict(X_test)

In [50]:
print('auc', roc_auc_score(y_test, y_pred))

auc 0.5763513513513514
